In [57]:
import os
import json
import random
import uuid
import asyncio

import gradio as gr
from dotenv import load_dotenv

In [58]:
try:
    import nest_asyncio
    nest_asyncio.apply()
except ImportError:
    raise ImportError("Run: pip install nest-asyncio")

In [59]:
try:
    from agents import Agent, Runner, function_tool
except ImportError:
    raise ImportError("Run: pip install openai-agents")


In [60]:
from openai import AsyncOpenAI
from agents.models.openai_chatcompletions import OpenAIChatCompletionsModel

In [61]:
OPENROUTER_API_KEY = os.getenv("API_TOKEN")

In [62]:
os.environ["OPENAI_AGENTS_DISABLE_TRACING"] = "1"

In [63]:
openrouter_client = AsyncOpenAI(
    api_key=OPENROUTER_API_KEY,
    base_url="https://openrouter.ai/api/v1",
    default_headers={
        "HTTP-Referer": "http://localhost:7860",
        "X-Title": "Interview Prep Agent",
    },
)

In [64]:
load_dotenv(override=True) 

True

In [65]:
# Model to use — swap to any OpenRouter model ID you like, e.g.:
#   "openai/gpt-4o"
#   "anthropic/claude-3.5-sonnet"
#   "mistralai/mixtral-8x7b-instruct"
#   "meta-llama/llama-3.1-70b-instruct"
MODEL = "meta-llama/llama-3.1-70b-instruct"

In [66]:
# ── Question Bank ─────────────────────────────────────────────────────────────
QUESTION_BANK: dict[str, dict[str, list[str]]] = {
    "Python Developer": {
        "Technical": [
            "Explain the difference between *args and **kwargs.",
            "What is the GIL and how does it affect multithreading?",
            "How does Python's memory management and garbage collection work?",
            "What are decorators and how do you create a custom one?",
            "Explain list comprehensions vs generator expressions.",
            "What is the difference between deepcopy and shallow copy?",
            "How does Python handle exceptions? Explain try/except/else/finally.",
            "What are context managers and how do you implement __enter__/__exit__?",
            "Explain the difference between @staticmethod and @classmethod.",
            "What are metaclasses in Python?",
        ],
        "Behavioral": [
            "Describe a time you debugged a particularly tricky Python issue.",
            "How do you keep up with new Python features and ecosystem changes?",
            "Tell me about a project where you improved Python code performance.",
            "How do you approach code reviews — giving and receiving feedback?",
            "Describe your strategy for writing maintainable, testable code.",
        ],
    },
    "Data Analyst": {
        "Technical": [
            "Explain the difference between INNER JOIN, LEFT JOIN, and FULL OUTER JOIN.",
            "How would you handle missing or null data in a dataset?",
            "What is the difference between correlation and causation?",
            "How would you detect and handle outliers in a dataset?",
            "What is a p-value and how do you interpret statistical significance?",
            "How would you design a dashboard for non-technical stakeholders?",
            "What are window functions in SQL? Give a practical example.",
            "How do you validate the accuracy and quality of your analysis?",
            "What is the difference between variance and standard deviation?",
            "Explain normalization vs denormalization in database design.",
        ],
        "Behavioral": [
            "Tell me about a time your analysis directly influenced a business decision.",
            "How do you explain complex data findings to non-technical audiences?",
            "Describe a situation where you discovered an unexpected insight.",
            "How do you prioritize when given multiple analysis requests at once?",
            "Tell me about a time you caught a significant error in an analysis.",
        ],
    },
    "Project Manager": {
        "Technical": [
            "What is the difference between Agile, Scrum, and Waterfall?",
            "How do you build and maintain a project risk register?",
            "Explain the critical path method in project scheduling.",
            "How do you define and track KPIs for a project?",
            "What tools do you use for project tracking and why?",
            "How do you manage scope creep effectively?",
            "What is earned value management (EVM) and how do you use it?",
            "How do you structure a work breakdown structure (WBS)?",
            "Explain the difference between a milestone and a deliverable.",
            "How do you manage task dependencies across teams?",
        ],
        "Behavioral": [
            "Tell me about a project that failed and what you learned from it.",
            "How do you handle conflict between team members?",
            "Describe a time you had to deliver difficult news to a stakeholder.",
            "How do you keep a team motivated when behind schedule?",
            "Tell me about managing competing priorities across multiple projects.",
        ],
    },
    "Frontend Developer": {
        "Technical": [
            "Explain the difference between CSS Flexbox and CSS Grid.",
            "What is the virtual DOM and how does React use it?",
            "How does event delegation work in JavaScript?",
            "What are closures in JavaScript and why are they useful?",
            "Explain the difference between == and === in JavaScript.",
            "What is CORS and how do you handle it on the frontend?",
            "Explain the browser's critical rendering path.",
            "What are Web Workers and when should you use them?",
            "What is tree shaking and how does it improve bundle size?",
            "Explain the difference between SSR, CSR, and SSG.",
        ],
        "Behavioral": [
            "Tell me about a challenging UI component you built.",
            "How do you balance design fidelity vs development speed?",
            "Describe a time you significantly improved page load performance.",
            "How do you ensure accessibility in your frontend work?",
            "Tell me about a time you disagreed with a designer's decision.",
        ],
    },
    "Machine Learning Engineer": {
        "Technical": [
            "Explain the bias-variance tradeoff.",
            "What is the difference between bagging and boosting?",
            "How do you handle class imbalance in training data?",
            "Explain gradient descent and its main variants (SGD, Adam, RMSProp).",
            "What is the difference between L1 and L2 regularization?",
            "What is k-fold cross-validation and why is it important?",
            "How would you deploy and monitor a machine learning model in production?",
            "What is feature engineering and why does it matter?",
            "Explain the attention mechanism in transformer architectures.",
            "What is the curse of dimensionality?",
        ],
        "Behavioral": [
            "Tell me about a model that performed well in training but failed in production.",
            "How do you explain ML model decisions to non-technical stakeholders?",
            "Describe your process for selecting a model architecture for a new problem.",
            "Tell me about a time you had to work with very messy, real-world data.",
            "How do you stay current with ML research and new techniques?",
        ],
    },
    "DevOps Engineer": {
        "Technical": [
            "Explain the key differences between containers and virtual machines.",
            "What is a CI/CD pipeline and how would you design one from scratch?",
            "How does Kubernetes handle service discovery and load balancing?",
            "What is infrastructure as code? Compare Terraform vs Ansible.",
            "Explain blue-green deployments vs canary deployments.",
            "How do you manage secrets in a cloud-native environment?",
            "What is the difference between horizontal and vertical scaling?",
            "How would you set up observability (logs, metrics, traces) for a service?",
            "Explain how a load balancer works and the types available.",
            "What are the main differences between AWS, GCP, and Azure?",
        ],
        "Behavioral": [
            "Tell me about a major production outage you responded to.",
            "How do you build a culture of reliability and operational excellence?",
            "Describe a significant manual process you automated.",
            "How do you handle pushback when enforcing security or compliance policies?",
            "Tell me about a deployment that went wrong and how you recovered.",
        ],
    },
}

In [67]:
ROLES = list(QUESTION_BANK.keys())

In [68]:
# ── In-memory session store (keyed by a simple session id) ────────────────────
# Each session: {"scores": [], "asked": set(), "role": str}
_SESSIONS: dict[str, dict] = {}

In [69]:
@function_tool
def get_question(session_id: str, role: str, category: str) -> str:
    """
    Fetch a fresh interview question for the given role and category.
    Avoids repeating questions already asked in this session.

    Args:
        session_id: Unique ID for this interview session.
        role:       Job role (e.g. 'Python Developer').
        category:   'Technical' or 'Behavioral'.
    Returns:
        A question string.
    """
    if session_id not in _SESSIONS:
        _SESSIONS[session_id] = {"scores": [], "asked": set(), "role": role}

    bank = QUESTION_BANK.get(role, {}).get(category, [])
    asked = _SESSIONS[session_id]["asked"]
    available = [q for q in bank if q not in asked] or bank  # reset if exhausted

    question = random.choice(available)
    _SESSIONS[session_id]["asked"].add(question)
    return question


In [70]:
@function_tool
def record_score(session_id: str, score: int) -> str:
    """
    Save a score (1–5) to the session tracker.

    Args:
        session_id: Unique ID for this interview session.
        score:      Integer 1 (poor) to 5 (excellent).
    Returns:
        Confirmation with running stats.
    """
    if session_id not in _SESSIONS:
        _SESSIONS[session_id] = {"scores": [], "asked": set(), "role": ""}

    _SESSIONS[session_id]["scores"].append(max(1, min(5, score)))
    scores = _SESSIONS[session_id]["scores"]
    avg = sum(scores) / len(scores)
    return f"Recorded {score}/5. Total: {len(scores)} answers, avg: {avg:.1f}/5."

In [71]:
@function_tool
def get_session_summary(session_id: str) -> str:
    """
    Return full performance summary for the session.

    Args:
        session_id: Unique ID for this interview session.
    Returns:
        JSON with scores, average, count, and performance label.
    """
    session = _SESSIONS.get(session_id)
    if not session or not session["scores"]:
        return json.dumps({"error": "No scores recorded yet."})

    scores = session["scores"]
    avg = sum(scores) / len(scores)
    label = (
        "Excellent" if avg >= 4.5 else
        "Great "     if avg >= 3.5 else
        "Good "      if avg >= 2.5 else
        "Fair "      if avg >= 1.5 else
        "Needs Work"
    )
    return json.dumps({
        "scores":          scores,
        "average":         round(avg, 2),
        "total_questions": len(scores),
        "performance":     label,
        "role":            session.get("role", ""),
    })

In [72]:
INSTRUCTIONS = """You are an Interview Coach conducting mock interviews.

TOOLS AVAILABLE:
- get_question(session_id, role, category): gets next interview question
- record_score(session_id, score): saves score 1-5
- get_session_summary(session_id): gets final stats

RULES — follow exactly:

On SESSION START (user gives role + session_id):
  1. Call get_question(session_id=<id>, role=<role>, category="Technical")
  2. Output ONLY this, nothing else:
  QUESTION: <the question text>
  QNUM: 1
  CATEGORY: Technical

On receiving an ANSWER:
  1. Score it 1-5. Call record_score(session_id=<id>, score=<n>)
  2. Call get_question for next question (alternate Technical/Behavioral)
  3. Output ONLY this, nothing else:
  SCORE: <1-5>
  STRENGTHS: <one sentence on what was good>
  IMPROVE: <one sentence on what to improve>
  IDEAL: <model answer in 2-3 sentences>
  QUESTION: <next question text>
  QNUM: <next number>
  CATEGORY: <Technical or Behavioral>

On STOP/QUIT/END:
  1. Call get_session_summary(session_id=<id>)
  2. Output ONLY this:
  SUMMARY: <encouraging 2-sentence summary>
  AVG: <average score>
  TOTAL: <number of questions>

Never add extra text. Never use JSON. Use the exact format above."""

In [73]:
def build_agent() -> Agent:
    return Agent(
        name="InterviewCoach",
        instructions=INSTRUCTIONS,
        tools=[get_question, record_score, get_session_summary],
        model=OpenAIChatCompletionsModel(
            model=MODEL,
            openai_client=openrouter_client,
        ),
    )

In [74]:
def run_agent_sync(agent: Agent, history: list[dict], user_msg: str) -> tuple[str, list[dict]]:
    """
    Run one agent turn synchronously.
    Works in both Jupyter (patched by nest_asyncio) and Gradio worker threads
    (which have no event loop — we create a fresh one for them).
    """
    history = history + [{"role": "user", "content": user_msg}]

    async def _run():
        return await Runner.run(agent, input=history)

    try:
        # Try getting existing loop first (works in Jupyter main thread)
        loop = asyncio.get_event_loop()
        if loop.is_closed():
            raise RuntimeError("Loop is closed")
        result = loop.run_until_complete(_run())
    except RuntimeError:
        # No event loop in this thread (Gradio worker thread) — create a new one
        result = asyncio.run(_run())

    reply = result.final_output
    return reply, history + [{"role": "assistant", "content": reply}]


In [75]:
def _get_field(text: str, key: str) -> str:
    """Extract a field value from the plain-text format: KEY: value"""
    for line in text.splitlines():
        if line.strip().upper().startswith(key.upper() + ":"):
            return line.split(":", 1)[1].strip()
    return ""

In [76]:
def parse_agent_reply(text: str) -> dict:
    """
    Parse agent reply in plain KEY: value format.
    Returns a dict with 'type' set to 'question', 'feedback', or 'summary'.
    """
    t = text.strip()
    has = lambda k: any(l.strip().upper().startswith(k+":") for l in t.splitlines())

    if has("SUMMARY") or has("AVG"):
        try:
            avg = float(_get_field(t, "AVG") or 0)
        except ValueError:
            avg = 0.0
        try:
            total = int(_get_field(t, "TOTAL") or 0)
        except ValueError:
            total = 0
        return {
            "type": "summary",
            "message": _get_field(t, "SUMMARY"),
            "average_score": avg,
            "total_questions": total,
            "performance": (
                "Excellent 🌟" if avg >= 4.5 else
                "Great 👍"     if avg >= 3.5 else
                "Good ✅"      if avg >= 2.5 else
                "Fair 📈"      if avg >= 1.5 else
                "Needs Work 💪"
            ),
        }

    if has("SCORE") and has("QUESTION"):
        try:
            score = int(_get_field(t, "SCORE") or 3)
        except ValueError:
            score = 3
        try:
            qnum = int(_get_field(t, "QNUM") or 1)
        except ValueError:
            qnum = 1
        return {
            "type": "feedback_and_question",
            "score": max(1, min(5, score)),
            "strengths":    _get_field(t, "STRENGTHS"),
            "improvements": _get_field(t, "IMPROVE"),
            "ideal_answer": _get_field(t, "IDEAL"),
            "question":     _get_field(t, "QUESTION"),
            "question_number": qnum,
            "category":     _get_field(t, "CATEGORY") or "Technical",
        }

    if has("QUESTION"):
        try:
            qnum = int(_get_field(t, "QNUM") or 1)
        except ValueError:
            qnum = 1
        return {
            "type": "question",
            "question": _get_field(t, "QUESTION"),
            "question_number": qnum,
            "category": _get_field(t, "CATEGORY") or "Technical",
        }

    # Fallback: return raw text as a question display
    return {"type": "raw", "text": t}

In [77]:
def fmt_feedback(p: dict) -> str:
    score = p.get("score", 0)
    stars = "⭐" * score + "☆" * (5 - score)
    return (
        f"### 📊 Score: {score}/5  {stars}\n\n"
        f"**✅ Strengths**\n{p.get('strengths', '—')}\n\n"
        f"**🔧 Areas to Improve**\n{p.get('improvements', '—')}\n\n"
        f"**💡 Ideal Answer**\n> {p.get('ideal_answer', '—')}"
    )

In [78]:
def fmt_question(p: dict) -> str:
    badge = "🔵 Technical" if p.get("category") == "Technical" else "🟣 Behavioral"
    return (
        f"**{badge}  |  Question {p.get('question_number', '?')}**\n\n"
        f"### 💬 {p.get('question', '')}"
    )

In [79]:
def fmt_summary(p: dict) -> str:
    avg = p.get("average_score", 0)
    filled = min(5, round(avg))
    stars = "⭐" * filled + "☆" * (5 - filled)
    return (
        f"## 🎉 Session Complete!  {stars}\n\n"
        f"**Score: {avg:.1f} / 5.0** — *{p.get('performance', '')}*\n\n"
        f"**Questions answered:** {p.get('total_questions', 0)}\n\n"
        f"{p.get('message', '')}\n\n---\n"
        f"*Click **New Session** to practice again!*"
    )


In [80]:
#  GRADIO HANDLERS
def start_interview(role, agent_state, api_history, session_id, scores):
    if not role:
        return (
            "⚠️ Please select a role first.", "",
            agent_state, api_history, session_id, scores,
            gr.update(interactive=False), gr.update(interactive=True),
        )
    try:
        sid = str(uuid.uuid4())[:8]
        agent = build_agent()
        raw, api_history = run_agent_sync(
            agent, [],
            f"Start a {role} interview. My session_id is '{sid}'."
        )
        parsed = parse_agent_reply(raw)
        if parsed["type"] == "question":
            question_md = fmt_question(parsed)
        else:
            question_md = f"### 💬 {parsed.get('question', raw)}"
        status = f"🎯 **{role}**  |  Questions: **0**  |  Avg: **—**"
        return (
            question_md, status,
            agent, api_history, sid, [],
            gr.update(interactive=True),
            gr.update(interactive=False),
        )
    except Exception as e:
        return (
            f"❌ **Error starting session:**\n```\n{e}\n```", "",
            agent_state, api_history, session_id, scores,
            gr.update(interactive=False), gr.update(interactive=True),
        )

In [81]:
def submit_answer(answer, agent_state, api_history, session_id, scores, chat_history):
    if not answer.strip():
        return chat_history, "", agent_state, api_history, session_id, scores, "", ""

    is_stop = any(w in answer.lower() for w in {"stop", "quit", "exit", "end"})

    # Gradio 6.6 requires dict messages: {"role": ..., "content": ...}
    chat_history = chat_history + [{"role": "user", "content": answer}]

    try:
        raw, api_history = run_agent_sync(
            agent_state, api_history,
            "end session" if is_stop else answer
        )
    except Exception as e:
        chat_history.append({"role": "assistant", "content": f"❌ Agent error:\n```\n{e}\n```"})
        return chat_history, "", agent_state, api_history, session_id, scores, "", ""

    parsed = parse_agent_reply(raw)

    # ── End of session ──
    if parsed["type"] == "summary":
        chat_history.append({"role": "assistant", "content": fmt_summary(parsed)})
        avg = parsed.get("average_score", 0)
        return (chat_history, "", agent_state, api_history, session_id, scores,
                f"✅ Done  |  Final avg: **{avg:.1f} / 5**", "")

    # ── Feedback + next question (combined reply) ──
    if parsed["type"] == "feedback_and_question":
        score = parsed.get("score", 0)
        scores = scores + [score]
        avg = sum(scores) / len(scores)
        chat_history.append({"role": "assistant", "content": fmt_feedback(parsed)})
        status = f"Questions: **{len(scores)}**  |  Avg: **{avg:.1f} / 5**"
        next_q_md = fmt_question(parsed)
        return chat_history, "", agent_state, api_history, session_id, scores, status, next_q_md

    # ── Raw / unrecognised response — show it so we can debug ──
    bot_text = parsed.get("text", raw) or "⚠️ No response parsed."
    chat_history.append({"role": "assistant", "content": bot_text})
    return chat_history, "", agent_state, api_history, session_id, scores, "", ""

In [82]:
def reset_session():
    return (
        [],   # chatbot — empty list, Gradio 6.6 handles dict format automatically
        None, [], "", [],
        "*Select a role and click Start Interview.*",
        "*Your question will appear here...*",
        gr.update(interactive=True),
        gr.update(interactive=False),
    )

In [83]:
#  GRADIO UI
CSS = """
#question-box {
    border-left: 4px solid #38bdf8;
    border-radius: 8px;
    padding: 14px;
    margin-bottom: 8px;
}
"""

with gr.Blocks(title="🎯 Interview Prep Agent") as demo:

    agent_state       = gr.State(None)
    api_history_state = gr.State([])
    session_id_state  = gr.State("")
    scores_state      = gr.State([])

    gr.Markdown("# 🎯 Interview Preparation Agent")
    gr.Markdown(
        "AI-powered mock interviews · real-time scoring · coaching feedback  \n"
        f"**Stack:** OpenAI Agents SDK + OpenRouter (free tier) + Gradio {gr.__version__}"
    )

    with gr.Row():

        with gr.Column(scale=1):
            gr.Markdown("### ⚙️ Setup")
            role_dropdown = gr.Dropdown(
                choices=ROLES, label="Select Job Role", value=None, interactive=True
            )
            start_btn = gr.Button("🚀 Start Interview", variant="primary")
            reset_btn = gr.Button("🔄 New Session",     variant="secondary")

            gr.Markdown("---")
            gr.Markdown("### 🛠️ Agent Tools")
            gr.Markdown(
                "**📚 get_question** — draws from curated bank, no repeats\n\n"
                "**📊 record_score** — persists scores per session\n\n"
                "**📋 get_session_summary** — final performance report"
            )
            gr.Markdown("---")
            gr.Markdown("### Score Guide")
            gr.Markdown(
                "**1** — Poor  \n **2** — Fair  \n"
                "**3** — Good  \n**4** — Great  \n"
                "**5** — Excellent"
            )

        with gr.Column(scale=3):
            status_bar       = gr.Markdown("*Select a role and click Start Interview.*")
            current_question = gr.Markdown(
                "*Your question will appear here...*", elem_id="question-box"
            )
            chatbot = gr.Chatbot(label="Interview Session", height=400)

            with gr.Row():
                answer_box = gr.Textbox(
                    placeholder="Type your answer... (Enter to submit, Shift+Enter for newline)",
                    label="Your Answer",
                    lines=3,
                    interactive=False,
                    scale=4,
                )
                submit_btn = gr.Button("Submit ➤", variant="primary", scale=1)

    # ── Events ────────────────────────────────────────────────────────────────
    start_btn.click(
        fn=start_interview,
        inputs=[role_dropdown, agent_state, api_history_state,
                session_id_state, scores_state],
        outputs=[current_question, status_bar, agent_state, api_history_state,
                 session_id_state, scores_state, answer_box, start_btn],
    )

    for trigger in [submit_btn.click, answer_box.submit]:
        trigger(
            fn=submit_answer,
            inputs=[answer_box, agent_state, api_history_state,
                    session_id_state, scores_state, chatbot],
            outputs=[chatbot, answer_box, agent_state, api_history_state,
                     session_id_state, scores_state, status_bar, current_question],
        )

    reset_btn.click(
        fn=reset_session,
        inputs=[],
        outputs=[chatbot, agent_state, api_history_state, session_id_state,
                 scores_state, status_bar, current_question, start_btn, answer_box],
    )

In [84]:
# ── Launch ────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    gr.close_all()
    print(f"✅ Gradio {gr.__version__}  |  Model: {MODEL}")
    demo.launch(
        server_name="127.0.0.1",
        server_port=7860,
        share=False,
        show_error=True,
        theme=gr.themes.Soft(),
        css=CSS,
    )


Closing server running on port: 7860
✅ Gradio 6.6.0  |  Model: meta-llama/llama-3.1-70b-instruct
* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
